设置requests_grad=True，追踪其上的所有操作

backgrad()求梯度

梯度保存在grad属性中

with torch.no_grad()将不想被追踪的代码块包裹起来

grad_fn 该tensor是怎么创建的

In [1]:
import torch

# 概念

Tensor是这个包的核心类，如果将其属性.requires_grad设置为True，它将开始追踪(track)在其上的所有操作（这样就可以利用链式法则进行梯度传播了）。完成计算后，可以**调用.backward()来完成所有梯度计算。此Tensor的梯度将累积到.grad属性中。**

注意：

在y.backward()时，如果y是标量，则不需要为backward()传入任何参数；否则，需要传入一个与y同形的Tensor。解释见 2.3.2 节。

如果不想要被继续追踪，可以调用.detach()将其从追踪记录中分离出来，这样就可以防止将来的计算被追踪，这样梯度就传不过去了。此外，还可以用with torch.no_grad()将不想被追踪的操作代码块包裹起来，这种方法在评估模型的时候很常用，因为在评估模型时，我们并不需要计算可训练参数（requires_grad=True）的梯度。

Function是另外一个很重要的类。Tensor和Function互相结合就可以构建一个记录有整个计算过程的有向无环图（DAG）。每个Tensor都有一个**.grad_fn属性，该属性即创建该Tensor的Function, 就是说该Tensor是不是通过某些运算得到的，若是，则grad_fn返回一个与这些运算相关的对象，否则是None。**

# Tensor

In [2]:
# requires_grad=True 追踪其上的所有操作
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)  # grad_fn属性，创建该Tensor的Function

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [3]:
y = x + 2
print(y)
print(y.grad_fn)  # grad_fn属性，创建该Tensor的Function，这里是通过一个加法操作创建的

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


直接创建的Tensor称为叶子节点，叶子节点对应的grad_fn是None

In [4]:
print(x.is_leaf, y.is_leaf)

True False


In [5]:
z = y * y * 3
out = z.mean()
print(z, out)  # mean操作得到该Tensor的

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


in_place的方式改变request_grad属性

In [6]:
a = torch.randn(2, 2)  # 默认requests_grad=False

a = ((a * 3) / (a - 1))
print(a.requires_grad)

a.requires_grad_(True)
print(a.requires_grad)

b = (a * a).sum()
print(b.grad_fn)  # sum操作得到该Tensor的

False
True


# 梯度

调用backward() 完成梯度计算

梯度累计到 .grad属性中

In [7]:
out.backward()  # 等价于 out.backward(torch.tensor(1.))

In [8]:
print(x.grad)  # grad属性，打印出梯度

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [9]:
print(x)
out2 = x.sum()
print(out2)
out2.backward()  # 完成梯度计算
print(x.grad)  # 打印出梯度，grad是累加的

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor(4., grad_fn=<SumBackward0>)
tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])


In [10]:
out3 = x.sum()
x.grad.data.zero_()  # 梯度清零
out3.backward()
print(x.grad)

tensor([[1., 1.],
        [1., 1.]])


In [11]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0],
                 requires_grad=True)  # 追踪其上的所有操作
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [12]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]],
                 dtype=torch.float)
z.backward(v)  # z求梯度，x的取值用v
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


In [13]:
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2

# 将不想被追踪的操作代码块包裹起来
# 这种方法在评估模型的时候很常用
# 因为在评估模型时，我们并不需要计算可训练参数（requires_grad=True）的梯度。
with torch.no_grad():
    y2 = x ** 3

y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad)
print(y2, y2.requires_grad)
print(y3, y3.requires_grad)

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [14]:
y3.backward()  # 求梯度
print(x.grad)  # 梯度保存在grad属性里

tensor(2.)


如果我们想要修改tensor的数值，但是又不希望被autograd记录（即不会影响反向传播），那么我么可以对tensor.data进行操作。

In [15]:
x = torch.ones(1, requires_grad=True)

print('x', x)
print(x.data)               # 还是一个tensor
print('x', x)
print(x.data.requires_grad) # 但是已经是独立于计算图之外

print('-' * 10)
y = 2 * x
print('x', x)
x.data *= 100  # 只改变了值，不会记录在计算图，所以不会影响梯度传播
print('x', x)  # x变了
print('x.data', x.data)

print('-' * 10)
y.backward()  # 梯度
print(x)
print(x.grad)  # 梯度保存在grad属性里

x tensor([1.], requires_grad=True)
tensor([1.])
x tensor([1.], requires_grad=True)
False
----------
x tensor([1.], requires_grad=True)
x tensor([100.], requires_grad=True)
x.data tensor([100.])
----------
tensor([100.], requires_grad=True)
tensor([2.])
